In [1]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# print("Num GPUs Available: ", len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [8]:
import os
#data_file_path = os.path.join(os.path.dirname(__file__), "YX.csv")
data_file_path = os.path.join(os.getcwd(), "Data\YX.csv")
#data_file_path = os.path.join(os.getcwd(), "Compustat.csv")

In [83]:
import pandas as pd
#finalData = pd.read_csv(data_file_path, nrows=10_000)
finalData = pd.read_csv(data_file_path)
finalData


# finalData = dataset.map(..., num_parallel_calls=10)
# finalData = dataset.prefetch(buffer_size)

,returns,gvkey,fdateq,datacqtr,curcdq,accdq,acoq,acoxq,actq,ancq,...,city,fyrc,ggroup,gind,gsector,gsubind,idbflag,loc,naics,sic
0,4.971591,1166,20100907,2010Q2,EUR,113.1330,55.8040,55.8040,817.555,274.2700,...,Almere,12,4530,453010,45,45301010,B,NLD,333242,3559
1,10.121786,1166,20101105,2010Q3,EUR,87.2820,70.5220,70.5220,839.315,253.3070,...,Almere,12,4530,453010,45,45301010,B,NLD,333242,3559
2,44.138609,1166,20110330,2010Q4,EUR,63.8670,67.9700,67.9700,934.149,279.9680,...,Almere,12,4530,453010,45,45301010,B,NLD,333242,3559
3,-37.643181,1166,20110811,2011Q2,EUR,284.1220,82.1470,82.1470,1216.409,345.5600,...,Almere,12,4530,453010,45,45301010,B,NLD,333242,3559
4,48.155340,1166,20120331,2011Q4,EUR,110.9440,97.0650,97.0650,1195.780,386.4410,...,Almere,12,4530,453010,45,45301010,B,NLD,333242,3559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59596,-7.707181,352723,20220903,2022Q2,CNY,174.2099,61.5137,61.4806,2523.141,203.3685,...,Shanghai,12,2010,201060,20,20106020,I,CHN,332912,3490
59597,8.107603,352948,20220821,2022Q1,INR,29.2730,71.4670,69.3530,1512.461,646.6230,...,Hyderabad,3,1510,151040,15,15104050,I,IND,331110,3312
59598,84.512350,353064,20220908,2022Q2,CNY,1354.9970,63.4908,50.9666,6807.913,1457.3440,...,Beijing,12,2510,251010,25,25101010,I,CHN,336320,3714
59599,-10.037170,353097,20220824,2022Q2,CNY,110.1364,13.1831,10.7148,1562.486,613.5332,...,Nanjing,12,3520,352030,35,35203010,I,CHN,325414,2836


In [ ]:
# import dask.dataframe
# finalData = dask.dataframe.read_csv(data_file_path)
# finalData
finalData.dtypes


testDF = pd.Dataframe({
    "A": [1, 2],
    "B": [6, 7]
}
)


In [89]:
# trainY = finalData["returns"][0:20].compute()
# trainY
# trainX = finalData.loc[:, finalData.columns != "returns"]
# trainX

import numpy as np

finalData["fdateq"] = pd.to_datetime(finalData["fdateq"], format = "%Y%m%d")
#finalData["fdateq"] = pd.to_datetime(finalData["datacqtr"], format = "%Y%q") #check quarter format


#Currently not grouped by quarter
finalData = finalData.sort_values(by=['datacqtr', "fdateq"])

trainSize = round(len(finalData.index)*0.4)
validationSize = round(len(finalData.index)*0.7)

Y = finalData["returns"]
Y = np.asarray(Y).astype('float32')
yTrain, yVal, yTest = Y[:trainSize], Y[trainSize:validationSize], Y[validationSize:]

X = finalData.loc[:, finalData.columns != "returns"]


factorCols = ["gvkey",  "sic", "naics", "loc", "idbflag",
              "gsubind", "ggroup", "gind", "gsector",
              "fic", "city", "fyrc",
              "curcdq", "exchg"]

X[factorCols] = X[factorCols].astype("str")



X = X.select_dtypes(include=[np.number])
XArray = np.asarray(X).astype('float32')

numberOfVariables = XArray.shape[1]




In [88]:
X.dtypes

finalData


,returns,gvkey,fdateq,datacqtr,curcdq,accdq,acoq,acoxq,actq,ancq,...,city,fyrc,ggroup,gind,gsector,gsubind,idbflag,loc,naics,sic
4029,-3.367876,200144,2010-02-18,2009Q4,ARS,134.5456,34.2808,13.2956,2228.9330,4987.7120,...,Buenos Aires,6,1510,151040,15,15104010,I,ARG,331313,3334
5176,-8.876812,201336,2010-02-22,2009Q4,ARS,342.5160,98.6690,63.8040,1151.5720,5216.6420,...,Buenos Aires,6,3020,302020,30,30202010,B,ARG,999977,9997
34651,0.628931,278800,2010-03-02,2009Q4,ARS,26.9737,6.7059,4.5902,440.8773,148.4538,...,Buenos Aires,12,3020,302020,30,30202030,I,ARG,3114,2030
9291,1.560847,206547,2011-02-12,2009Q4,HUF,586.0000,1234.0000,1234.0000,86717.0000,68975.0000,...,Budapest,9,3520,352020,35,35202010,I,HUN,325412,2834
38365,-33.637890,285237,2020-12-04,2009Q4,EUR,5.5242,0.6135,0.0573,21.1166,13.8050,...,Aigrefeuille-d'Aunis,8,2520,252020,25,25202010,I,FRA,336612,3730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58036,-12.051230,325357,2022-08-20,2022Q2,CNY,745.0450,58.2354,16.3817,10809.6900,2933.2940,...,Guangzhou,12,3510,351020,35,35102015,I,CHN,6215,8071
32981,3.551653,274159,2022-08-20,2022Q2,TWD,587.6160,48.5040,6.6760,5842.6520,2146.1050,...,Zhubei,12,4520,452020,45,45202030,I,TWN,334413,3674
20266,16.072100,246076,2022-08-20,2022Q2,TWD,45.3430,128.9580,0.6170,6104.5320,1049.9450,...,Taipei,12,2010,201070,20,20107010,I,TWN,325,2800
21405,67.173140,247969,2022-08-19,2022Q2,CNY,533.2028,233.9526,233.9526,3588.2920,1388.4280,...,Shanghai,12,4530,453010,45,45301020,I,HKG,334413,3674


In [90]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_X = scaler.fit_transform(XArray)
#scaled_X = np.apply_along_axis(func1d=scaler.fit_transform, axis=1, arr=X)

xTrain, xVal, xTest = scaled_X[:trainSize,:], scaled_X[trainSize:validationSize,:], scaled_X[validationSize:, :]




#scaled_X = scaled_X.apply(lambda x : x.reshape(-1, 1))
# droppedCols = ["ajexdi", "prccd", "trfd"]
# trainX.drop(droppedCols, axis=1, inplace=True)
# trainX['gvkey'] = trainX['gvkey'].astype(object)
# trainX['gvkey'] = trainX['gvkey'].astype(object)

#trainX.dtypes

In [91]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# print("Num GPUs Available: ", len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [46]:
# model = Sequential([
#     Dense(units=16,  activation='relu'), #input_shape=(, 10, numberOfVariables),
#     Dense(units=32, activation='relu'),
#     Dense(units=1)
# ])


In [47]:
# model = tf.keras.Sequential()

# model.add(Dense(units=10,  activation='relu'))
# model.add(tf.keras.layers.Dropout(.5))
# model.add(Dense(units=10, activation='relu'))
# model.add(Dense(units=1))

In [48]:
# model.compile(optimizer="sgd",  # Adam(learning_rate=0.0001),
#               loss="MeanSquaredError", #'tf.keras.losses.MeanSquaredError()',
#               metrics=['accuracy'])

In [49]:
# model.fit(x=xTrain, y=yTrain,
#           batch_size=10, epochs=30, verbose=2)


In [50]:
# loss_and_metrics = model.evaluate(xVal, yVal, batch_size=128)

# model.predict(xVal, batch_size=128)

In [92]:
def NNfunction(modelType, layers, hidUnits, actvFunc, dropout, L1, L2, epo):
    
    model = tf.keras.Sequential()

    if modelType == "LSTM":
        model.add(tf.keras.layers.LSTM(units=hidUnits, input_shape=(2,)))
    elif modelType == "GRU":
        model.add(tf.keras.layers.GRU(units=hidUnits))
    elif modelType == "SimpleRNN":
        # keras.layers.SimpleRNN(1, input_shape=[None, 1])
        model.add(tf.keras.layers.SimpleRNN(units=hidUnits))
    elif modelType == "Linear":
        model.add(Dense(units=1,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=L1, l2=L2)))   #insert lasso and ridge here
    else:
        model.add(Dense(units=hidUnits,
                        activation=actvFunc,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=L1, l2=L2)))
        try: 
            for n in range(0, layers):
                model.add(Dense(units=hidUnits, activation=actvFunc))
                model.add(tf.keras.layers.Dropout(dropout))
        except:
            model.add(Dense(units=hidUnits, activation=actvFunc))
            model.add(tf.keras.layers.Dropout(dropout))

            
        model.add(Dense(units=1))



    model.compile(optimizer="sgd",  # Adam(learning_rate=0.0001),
                  loss="MeanSquaredError",  # 'tf.keras.losses.MeanSquaredError()',
                  metrics=['accuracy'])


    model.fit(x=xTrain, y=yTrain,
              batch_size=16, epochs=epo, verbose=2)


    loss_and_metrics = model.evaluate(xVal, yVal, batch_size=128)
    try:
        loss_and_metrics = [round(x) for x in loss_and_metrics]
    except:     
        print(loss_and_metrics)

    return loss_and_metrics, model

    #model.predict(xVal, batch_size=128)

    





In [93]:
#Hyper-Parameters
modelType = ["Linear", "Forward", "LSTM", "GRU"]
hiddenLayers = [2, 3]
hiddenUnits = [2, 3]
activationFunc = ["relu", "sigmoid"] #softmax, etc. https://keras.io/api/layers/activations/
dropOut = [0.5, 0]
#regGrid = [10^seq(3,-10, length=4), 0]
epochs = [1] #Just choose best model. => Run for 5 epochs, but if 3rd epoch is better, choose it
inputSize = numberOfVariables #ncol(trainXPools[[1]])

# import numpy as np
# NNParams = np.array([(modelType, layers, hidUnits, actvFunc, dropout, epo)
#               for modelType in ["Forward"]
#               for layers in hiddenLayers
#               for hidUnits in hiddenUnits
#               for actvFunc in activationFunc
#               for dropout in dropOut
#               for epo in epochs])
#             #   for l1 in regGrid
#             #   for l2 in regGrid


# RNNParams = np.array([(modelType, layers, hidUnits, actvFunc, dropout, epo)
#               for modelType in ["LSTM", "GRU"]
#               for layers in hiddenLayers
#               for hidUnits in hiddenUnits
#               for actvFunc in ["NA"]
#               for dropout in dropOut
#               for epo in epochs])

# LinearParams = np.array([(modelType, layers, hidUnits, actvFunc, dropout, epo)
#               for modelType in ["Linear"]
#               for layers in [0]
#               for hidUnits in [0]
#               for actvFunc in ["NA"]
#               for dropout in [0]
#               for epo in [1]])



L1L2Grid = 10**np.linspace(start=2, stop=-10, num=2)
L1L2Grid = np.append(L1L2Grid, 0)


NNParams = {"modelType": ["Forward"],#, "LSTM", "GRU"],
               "hiddenLayers": [2, 3, 4],
               "hiddenUnits": [2, 3, 4, 5, 10],
               "activationFunc": ["relu", "sigmoid"],
               "dropOut": [0.5, 0.2, 0],
               "L1": L1L2Grid,
               "L2": L1L2Grid,
               "epochs": [2]
               }

LinearParams = {"modelType": ["Linear"],
               "hiddenLayers": ["NA"],
               "hiddenUnits": ["NA"],
               "activationFunc": ["NA"],
               "dropOut": ["NA"],
               "L1": L1L2Grid,
               "L2": L1L2Grid,
               "epochs": [2]
               }



import itertools
import pandas as pd
NNGrid = pd.DataFrame(itertools.product(
    *NNParams.values()), columns=NNParams.keys())

linearGrid = pd.DataFrame(itertools.product(
    *LinearParams.values()), columns=LinearParams.keys())


#Random grid search
#import random
#random.seed(2302)
randomNNGrid = NNGrid.sample(n=5)

HPGrid = pd.concat([linearGrid, randomNNGrid])


# hyperParams = np.concatenate((LinearParams, NNParams, RNNParams))
#hyperParams = np.concatenate(grid, LinearParams)

validationSize = 2 #length(validXPools)
#HPsize = len(hyperParams.index)
HPsize = np.shape(HPGrid)[0]





In [94]:
MSEMatrix = pd.DataFrame()
pd.options.mode.chained_assignment = None
HPsize = 12

for n in range(0, HPsize):
  print("\n")
  print("Model ", str(n+1), " out of ", str(HPsize))

  HP = HPGrid.iloc[n]
  CV = NNfunction(*HP)

  #a = pd.DataFrame([CV])
  newMSE = HPGrid.iloc[[n]]

  MSE = CV[0][0]
  MAE = CV[0][1]


  # newMSE['MSE'] = MSE
  # newMSE['MAE'] = MAE


  newMSE.loc[:,"MSE"] = MSE
  newMSE.loc[:,"MAE"] = MAE

  MSEMatrix = pd.concat((MSEMatrix, newMSE), axis=0)
  
  # newMSE = a.join(b)
  # newMSE2 = pd.concat([b, a])
  #newMSE = pd.concat([b, a], axis=1, ignore_index=True, sort=False)
  #newMSE =  pd.concat(HPGrid.iloc[[n]], CV)
  #newMSE = np.concatenate((HP, CV))
  #MSEMatrix = np.concatenate((MSEMatrix, newMSE))
  #MSEMatrix = pd.concat(MSEMatrix, newMSE)
  #MSEMatrix = pd.concat([MSEMatrix, [HP, CV]])

MSEMatrix
#finalResults = MSEMatrix
#finalResults[:,-1] = apply(finalResults[,-1], MARGIN=2, FUN=as.numeric)
#finalResults = finalResults[order(finalResults[,"MSE"]),]
#finalResults <- apply(finalResults, 2, as.character)

#write.csv2(finalResults, "results3.csv", row.names = FALSE)


#pd.DataFrame.to_csv(MSEMatrix, "resultsPython.csv")







Model  1  out of  12
Epoch 1/2
1490/1490 - 3s - loss: nan - accuracy: 0.0105 - 3s/epoch - 2ms/step
Epoch 2/2
1490/1490 - 2s - loss: nan - accuracy: 0.0161 - 2s/epoch - 1ms/step
140/140 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.0084
[nan, 0.008388792164623737]


Model  2  out of  12
Epoch 1/2
1490/1490 - 3s - loss: 6341.0078 - accuracy: 2.0973e-04 - 3s/epoch - 2ms/step
Epoch 2/2
1490/1490 - 2s - loss: 6340.5156 - accuracy: 8.3893e-05 - 2s/epoch - 1ms/step
140/140 [==============================] - 0s 2ms/step - loss: 6015.1743 - accuracy: 1.6778e-04


Model  3  out of  12
Epoch 1/2
1490/1490 - 3s - loss: 6337.3130 - accuracy: 2.5168e-04 - 3s/epoch - 2ms/step
Epoch 2/2
1490/1490 - 2s - loss: 6338.8608 - accuracy: 8.3893e-05 - 2s/epoch - 1ms/step
140/140 [==============================] - 0s 2ms/step - loss: 6185.8545 - accuracy: 1.6778e-04


Model  4  out of  12
Epoch 1/2
1490/1490 - 3s - loss: nan - accuracy: 0.0111 - 3s/epoch - 2ms/step
Epoch 2/2
1490/1

,modelType,hiddenLayers,hiddenUnits,activationFunc,dropOut,L1,L2,epochs,MSE,MAE
0,Linear,NA,NA,NA,NA,1.000000e+02,1.000000e+02,2,NaN,0.008389
1,Linear,NA,NA,NA,NA,1.000000e+02,1.000000e-10,2,6015.0,0.000000
2,Linear,NA,NA,NA,NA,1.000000e+02,0.000000e+00,2,6186.0,0.000000
3,Linear,NA,NA,NA,NA,1.000000e-10,1.000000e+02,2,NaN,0.008389
4,Linear,NA,NA,NA,NA,1.000000e-10,1.000000e-10,2,3038.0,0.000000
5,Linear,NA,NA,NA,NA,1.000000e-10,0.000000e+00,2,3226.0,0.000000
6,Linear,NA,NA,NA,NA,0.000000e+00,1.000000e+02,2,NaN,0.008389
7,Linear,NA,NA,NA,NA,0.000000e+00,1.000000e-10,2,3055.0,0.000000
8,Linear,NA,NA,NA,NA,0.000000e+00,0.000000e+00,2,3115.0,0.000000
735,Forward,4,5,sigmoid,0.5,0.000000e+00,1.000000e+02,2,3884.0,0.000000


In [95]:

#MSEMatrix

index = MSEMatrix['MSE'].idxmax()
bestParams = MSEMatrix.iloc[index][:-2]
bestModel = NNfunction(*bestParams)[1]



bestModel.save('Results/bestKerasModel')

#model = keras.models.load_model('Results/bestKerasModel')




Epoch 1/2
1490/1490 - 5s - loss: 6338.2437 - accuracy: 2.5168e-04 - 5s/epoch - 3ms/step
Epoch 2/2
1490/1490 - 3s - loss: 6341.4897 - accuracy: 1.6779e-04 - 3s/epoch - 2ms/step
140/140 [==============================] - 1s 3ms/step - loss: 5972.2061 - accuracy: 1.6778e-04
INFO:tensorflow:Assets written to: Results/bestKerasModel\assets


In [ ]:
resPath = "C:\\Users\RobinForMLThesis\\OneDrive - Hanken Svenska handelshogskolan\\Master's_Thesis\\DataAnalysis\\resultsPython.csv"

finalResults = pd.DataFrame(MSEMatrix)
pd.DataFrame.to_csv(finalResults, resPath)
#pd.DataFrame.to_csv(finalResults, "resultsPython.csv")

MSEMatrix.tofile(resPath, sep = ',')

finalResults

In [ ]:
#ynew = model.predict_classes(Xnew)

In [ ]:
#(Geron 2019, 76, 320)

#from sklearn import RandomizedSearchCV, GridSearchCV



#(Geron 2019, 76)
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)



grid_search.best_params_

grid_search.best_estimator_

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)





#(Geron 2019, 320)


def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
        model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model


keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


keras_reg.fit(X_train, y_train, epochs=100,
validation_data=(X_valid, y_valid),
callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)




from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
validation_data=(X_valid, y_valid),
callbacks=[keras.callbacks.EarlyStopping(patience=10)])


rnd_search_cv.best_params_
rnd_search_cv.best_score_
model = rnd_search_cv.best_estimator_.model








#Chollet: DL for Python
import kerastuner as kt





#strategy = tf.distribute.MirroredStrategy()





#If your search process crashes, you can always restart it—just specify overwrite=False in the tuner so that it can resume from the trial logs stored on disk.




